In [9]:
from sqlalchemy import create_engine
import pandas as pd

from pandas.tseries.holiday import *
from pandas.tseries.holiday import Holiday, sunday_to_monday, Easter, Day
from pandas.tseries.offsets import CustomBusinessDay

In [10]:
user = "schedulin"
passw = "MySQLIsFun"
host = "35.231.228.133"
database = "schedulin"

In [11]:
db = create_engine(
    'mysql+pymysql://{0}:{1}@{2}/{3}' \
        .format(user, passw, host, database), \
    connect_args = {'connect_timeout': 10})
conn = db.connect()

In [12]:
timetable_df = pd.read_sql_table(
    "timetable",
    con = conn)
resource_df = pd.read_sql_table(
    "resource",
    con = conn)
user_df = pd.read_sql_table(
    "user",
    con = conn)
reservation_df = pd.read_sql_table(
    "reservation",
    con = conn)

In [13]:
merge_1_df = resource_df.merge(timetable_df, left_on = "timetable_id", right_on = "id", suffixes=["_res", "_tim"])\
    .drop(axis = 1, columns = [
        "creation_date_res", "modification_date_res", "id_tim"])\
    .rename(columns={"id_res": "resource_id"})

In [14]:
merge_2_df = reservation_df.merge(user_df, left_on = "user_id", right_on = "id", suffixes=["_rva", "_use"])\
    .drop(axis = 1, columns = [
        "creation_date_rva", "modification_date_rva",
        "creation_date_use", "modification_date_use", "id_use"])\
    .rename(columns={"id_rva": "reservation_id"})

In [15]:
merge_3_df = merge_2_df.merge(merge_1_df, on = "resource_id",
                    suffixes = ["_rva", ""])\
    .drop(axis = 1, columns = [
        "deletion_date_rva", "deletion_date_res",
        "deletion_date_tim"])

In [16]:
import numpy as np

merge_3_df["is_legit"] = np.select([
    merge_3_df["date"] > \
        merge_3_df["deletion_date_use"]],
    [False],
    default = True
)

In [17]:
model_df = merge_3_df.groupby(["date"]).sum()[["is_legit"]].reset_index()
model_df

,date,is_legit
0,2012-11-04,11
1,2012-11-05,6
2,2012-11-06,7
3,2012-11-07,6
4,2012-11-08,11
...,...,...
3648,2022-10-31,49
3649,2022-11-01,56
3650,2022-11-02,51
3651,2022-11-03,56


In [18]:
model_df["day_of_week"] = model_df["date"].dt.day_name()
model_df["month"] = model_df["date"].dt.month_name()

In [19]:
def convert_to_season(column):
    if column in ["January","February","December"]:
        return "Winter"
    elif column in ["March","April","May"]:
        return "Spring"
    elif column in ["June","July","August"]:
        return "Summer"
    else:
        return "Autumn"

In [20]:
model_df["season"] = model_df["month"].apply(convert_to_season)

In [21]:
model_df["week_day"] = model_df["day_of_week"].apply(lambda x: "weekday" if x == "Monday" or x == "Tuesday"
                                                        or x == "Wednesday" or x == "Thursday" or x == "Friday"
                                                        else "weekend")

In [22]:
model_df.head()

,date,is_legit,day_of_week,month,season,week_day
0,2012-11-04,11,Sunday,November,Autumn,weekend
1,2012-11-05,6,Monday,November,Autumn,weekday
2,2012-11-06,7,Tuesday,November,Autumn,weekday
3,2012-11-07,6,Wednesday,November,Autumn,weekday
4,2012-11-08,11,Thursday,November,Autumn,weekday


In [23]:
import pandas as pd
df = pd.read_csv("temperature.csv")

In [24]:
df

,Unnamed: 0,time,apparent_temperature_max (°C),precipitation_sum (mm)
0,335,2012-12-01,3.3,0.0
1,336,2012-12-02,5.7,0.0
2,337,2012-12-03,7.3,0.0
3,338,2012-12-04,8.5,0.0
4,339,2012-12-05,6.0,0.0
...,...,...,...,...
3507,3842,2022-07-09,37.5,0.0
3508,3843,2022-07-10,38.4,0.0
3509,3844,2022-07-11,38.1,0.6
3510,3845,2022-07-12,37.7,0.0


In [25]:
df["time"] = pd.to_datetime(df["time"])

In [26]:
merge = model_df.merge(df, right_on="time", left_on="date")

In [27]:
merge.head()

,date,is_legit,day_of_week,month,season,week_day,Unnamed: 0,time,apparent_temperature_max (°C),precipitation_sum (mm)
0,2012-12-01,22,Saturday,December,Winter,weekend,335,2012-12-01,3.3,0.0
1,2012-12-02,19,Sunday,December,Winter,weekend,336,2012-12-02,5.7,0.0
2,2012-12-03,9,Monday,December,Winter,weekday,337,2012-12-03,7.3,0.0
3,2012-12-04,6,Tuesday,December,Winter,weekday,338,2012-12-04,8.5,0.0
4,2012-12-05,13,Wednesday,December,Winter,weekday,339,2012-12-05,6.0,0.0


In [28]:
merge = merge.drop(axis=1, columns=["Unnamed: 0","time"])

In [29]:
merge = merge.rename(columns={"is_legit":"reservations",
                    "apparent_temperature_max (°C)":"temp", "precipitation_sum (mm)":"prec"})

In [30]:
merge.head()

,date,reservations,day_of_week,month,season,week_day,temp,prec
0,2012-12-01,22,Saturday,December,Winter,weekend,3.3,0.0
1,2012-12-02,19,Sunday,December,Winter,weekend,5.7,0.0
2,2012-12-03,9,Monday,December,Winter,weekday,7.3,0.0
3,2012-12-04,6,Tuesday,December,Winter,weekday,8.5,0.0
4,2012-12-05,13,Wednesday,December,Winter,weekday,6.0,0.0


In [31]:
date_range = pd.date_range("2012-12-01", "2022-07-13")

In [32]:
#Comprobando que no hay fechas missing
print(len(date_range))
print(len(merge))

3512
3512


In [33]:
class EsBusinessCalendar(AbstractHolidayCalendar):
   rules = [
     Holiday('Año Nuevo', month=1, day=1, observance=sunday_to_monday),
     Holiday('Epifanía del Señor', month=1, day=6, observance=sunday_to_monday),
     Holiday('Jueves Santo', month=1, day=1, offset=[Easter(), Day(-3)]),
     Holiday('Viernes Santo', month=1, day=1, offset=[Easter(), Day(-2)]),
     Holiday('Lunes de Pascua', month=1, day=1, offset=[Easter(), Day(+1)]),
     Holiday('Día del Trabajador', month=5, day=1, observance=sunday_to_monday),
     Holiday('San Isidro', month=5, day=15, observance=sunday_to_monday),
     Holiday('Día de la Comunidad de Madrid', month=5, day=2, observance=sunday_to_monday),
     Holiday('Santiago Apóstol', month=7, day=25, observance=sunday_to_monday),
     Holiday('Asunción de la Virgen', month=8, day=15, observance=sunday_to_monday),
     Holiday('Día de la Hispanidad', month=10, day=12, observance=sunday_to_monday),
     Holiday('Todos los Santos', month=11, day=1, observance=sunday_to_monday),
     Holiday('Nuestra Señora de la Almudena', month=11, day=9, observance=sunday_to_monday),
     Holiday('Día de la Constitución Española', month=12, day=6, observance=sunday_to_monday),
     Holiday('Inmaculada Concepción', month=12, day=8, observance=sunday_to_monday),	    
     Holiday('Navidad', month=12, day=25, observance=sunday_to_monday),
     Holiday('Nochevieja', month=12, day=31, observance=sunday_to_monday)

   ]

es_BD = CustomBusinessDay(calendar=EsBusinessCalendar())
s = pd.date_range('2012-11-03', end='2022-11-05', freq=es_BD)
businessDay_df = pd.DataFrame(s, columns=['Fecha'])
print(businessDay_df)

          Fecha
0    2012-11-05
1    2012-11-06
2    2012-11-07
3    2012-11-08
4    2012-11-12
...         ...
2458 2022-10-28
2459 2022-10-31
2460 2022-11-02
2461 2022-11-03
2462 2022-11-04

[2463 rows x 1 columns]


In [34]:
businessDay_df['isbankholiday'] = 0
new_reservation_df = merge.merge(businessDay_df,left_on = "date", right_on='Fecha',how="left")
mask = new_reservation_df['isbankholiday'].isnull()
new_reservation_df['isbankholiday'][mask] = 1

c:\Users\plafu\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [35]:
new_reservation_df['isbankholiday'].value_counts()


0.0    2366
1.0    1146
Name: isbankholiday, dtype: int64

In [36]:
new_reservation_df.head(5)

,date,reservations,day_of_week,month,season,week_day,temp,prec,Fecha,isbankholiday
0,2012-12-01,22,Saturday,December,Winter,weekend,3.3,0.0,NaT,1.0
1,2012-12-02,19,Sunday,December,Winter,weekend,5.7,0.0,NaT,1.0
2,2012-12-03,9,Monday,December,Winter,weekday,7.3,0.0,2012-12-03,0.0
3,2012-12-04,6,Tuesday,December,Winter,weekday,8.5,0.0,2012-12-04,0.0
4,2012-12-05,13,Wednesday,December,Winter,weekday,6.0,0.0,2012-12-05,0.0
